Handle Evolving Workflows
=========================

For some workflows we don't know the extent of the computation at the outset.  We need to do some computation in order to figure out the rest of the computation that we need to do.  The computation grows and evolves as we do more work.

As an example, consider a situation where you need to read many files and then based on the contents of those files, fire off additional work.  You would like to read the files in parallel, and then within each file expose more parallelism.

This example goes through three ways to handle this situation using [Dask Futures](https://docs.dask.org/en/latest/futures.html)

1.  Using `as_completed`
2.  Using `async/await`
3.  Launching tasks from tasks

But first, lets run our code sequentially.

0: Sequential code
------------------

In [1]:
filenames = ["file.{}.txt".format(i) for i in range(10)]

filenames[:3]

['file.0.txt', 'file.1.txt', 'file.2.txt']

In [2]:
import random, time


def parse_file(fn: str) -> list:
    """ Returns a list work items of unknown length """
    time.sleep(random.random())
    return [random.random() for _ in range(random.randint(1, 10))]

def process_item(x: float):
    """ Process each work item """
    time.sleep(random.random() / 4)
    return x + 1

In [3]:
%%time

# This takes around 10-20s

results = []

for fn in filenames:
    L = parse_file(fn)
    for x in L:
        out = process_item(x)
        results.append(out)

CPU times: user 4.37 ms, sys: 668 µs, total: 5.03 ms
Wall time: 13.7 s


Start Dask Client
-----------------

We'll need a Dask client in order to manage dynamic workloads

In [4]:
from dask.distributed import Client

client = Client(processes=False, n_workers=1, threads_per_worker=6)
client

Client Scheduler: inproc://10.1.0.4/7612/1 Dashboard: http://10.1.0.4:8787/status,Cluster Workers: 1 Cores: 6 Memory: 7.29 GB


1: Use as_completed
-------------------

The [as_completed](https://docs.dask.org/en/latest/futures.html#distributed.as_completed) iterator lets us handle futures as they complete.  We can then submit more data on the fly.

-   We submit a task for each of our filenames
-   We also compute the length of each of the returned lists
-   As those lengths return, we submit off a new task to get each item of that list.  We do this at higher priority, so that we process existing data before we collect new data.
-   We wait on all of the returned results

In [5]:
%%time

from dask.distributed import as_completed
import operator

lists = client.map(parse_file, filenames, pure=False)
lengths = client.map(len, lists)

mapping = dict(zip(lengths, lists))

futures = []

for future in as_completed(lengths):
    n = future.result()
    L = mapping[future]
    for i in range(n):
        new = client.submit(operator.getitem, L, i, priority=1)
        new = client.submit(process_item, new, priority=1)
        futures.append(new)
        
client.gather(futures)

CPU times: user 279 ms, sys: 15.8 ms, total: 295 ms
Wall time: 2.29 s


[1.6408141464097064,
 1.769424768671146,
 1.9378567629768635,
 1.4650335727990513,
 1.8824065165131387,
 1.7829365319603308,
 1.2344454366032487,
 1.5485120232381808,
 1.7841997989682876,
 1.989308959491595,
 1.8530056462007507,
 1.7599532075279507,
 1.5499626833150635,
 1.3317196813807302,
 1.9482902103385238,
 1.3173361892999342,
 1.315034767676297,
 1.2323896472139828,
 1.079142830440663,
 1.7820589354515788,
 1.6522424639993474,
 1.2472176511852422,
 1.9879056717798913,
 1.836345866401777,
 1.1031364397412893,
 1.667327885958536,
 1.5687889651086073,
 1.887726397815149,
 1.214771176841841,
 1.5488577052889259,
 1.516566036025409,
 1.3872412889917156,
 1.939533189565965,
 1.5865850367916305,
 1.2235092266830936,
 1.6978729358097597,
 1.0469938404189008,
 1.1283280730672431,
 1.5449254718888756,
 1.9755802337845576,
 1.388375073191901,
 1.8970226956313723,
 1.3359912806911383,
 1.8467914549065276,
 1.1638011127459658,
 1.7848805698801389,
 1.965613745835093,
 1.7020374000744791,
 1.5

2: Use async/await to handle single file processing locally
-----------------------------------------------------------

We can also handle the concurrency here within our local process.  This requires you to understand async/await syntax, but is generally powerful and arguably simpler than the `as_completed` approach above.

In [6]:
import asyncio

async def f(fn):
    """ Handle the lifecycle of a single file """
    future = client.submit(parse_file, fn, pure=False)
    length_future = client.submit(len, future)
    length = await length_future
    
    futures = [client.submit(operator.getitem, future, i, priority=10) 
               for i in range(length)]
    futures = client.map(process_item, futures, priority=10)
    return futures

async def run_all(filenames):
    list_of_list_of_futures = await asyncio.gather(*[f(fn) for fn in filenames])
    futures = sum(list_of_list_of_futures, [])
    return await client.gather(futures)


We now need to run this function in the same event loop as our client is running.  If we had started our client asynchronously, then we could have done this:

```python
client = await Client(asynchronous=True)

await run_all(filenames)
```

However, because we started our client without the `asynchronous=True` flag the event loop is actually running in a separate thread, so we'll have to ask the client to run this for us.

In [7]:
client.sync(run_all, filenames)

[1.908142248332966,
 1.1369870531082809,
 1.66838297549333,
 1.178929176129981,
 1.0484648350852064,
 1.9014409116851125,
 1.945069205479884,
 1.6982667489139147,
 1.3075243046660479,
 1.4066560074041883,
 1.6277312264413017,
 1.0141164097160797,
 1.837273626855097,
 1.8555540972205264,
 1.2322907780074739,
 1.4012707979415047,
 1.9024480283805003,
 1.0037167432399956,
 1.4412852164382275,
 1.721280221561668,
 1.7021386254561999,
 1.253490738844739,
 1.5253527081299498,
 1.1175030329615268,
 1.4740813362824705,
 1.626120806896337,
 1.3960272696794869,
 1.3770989300352625,
 1.9827829952892897,
 1.9036717707063486,
 1.5562983750122186,
 1.1726812995075004,
 1.4939178939141122,
 1.3516958516805988,
 1.638596201426845,
 1.0607659619789636,
 1.213346415747526,
 1.7311303803583415,
 1.6184184281872804,
 1.9266201579757212,
 1.036799155539557,
 1.0075311847844355,
 1.8637086523506259,
 1.8457385023540456,
 1.238080070927448,
 1.9593506721834277,
 1.6503731054811805,
 1.7862656694514436,
 1.65

3: Submit tasks from tasks
--------------------------

We can also submit tasks that themselves submit more tasks.  See [documentation here](https://docs.dask.org/en/latest/futures.html#submit-tasks-from-tasks).

In [8]:
%%time

from dask.distributed import get_client, secede, rejoin

def f(fn):
    L = parse_file(fn)
    client = get_client()
    
    futures = client.map(process_item, L, priority=10)
    secede()
    results = client.gather(futures)
    rejoin()
    return results

futures = client.map(f, filenames, pure=False)
results = client.gather(futures)

CPU times: user 208 ms, sys: 11.2 ms, total: 219 ms
Wall time: 2.69 s
